# Jupyter PDF Table Data Analysis
<div style="text-align: justify">
The purpose of this project is to take real action on a data-driven project. The project isn't complex at all, but it's sufficient for demonstrating my original intention to apply some basic Statistics and Probability Theory to real-world data. Additionally, by taking real action, I anticipate to encounter more challenges, which will allow even the simpler project to showcase various aspects of my skills and my problem-solving capabilities.

## Environment Management
Project Environment Name: Ham_Col_Analysis  
Project Directory Name: ham_col_analysis_prj  

In [ ]:
# Ensure Jupiter is working at the correct environment to avoid conflicts:
import sys
sys.executable

## Table Exctraction from PDF
<div style="text-align: justify">
The table of interest is the B1 PDF table and located at page 3. Tabula library's results weren't as good as camelot library's results. For this reason, I chose camelot for the pdf's page 3 table extraction. This library also requires PyPDF2 library which is a library for text extraction from pdfs. However, PyPDF2 can't extract the tables separately from the text and hence it doesn't offer a solution on its own for this kind of problems. Additionally, the ghostscript library was also required for camelot to work properly. Some downgrades were also needed in order to resolve internal camelot conflicts.

In [1]:
import pandas as pd

# Libraries for PDF data extraction:
import camelot
import PyPDF2
import ghostscript

In [2]:
# Relative path to the pdf file:
pdf_path = "CDS_2017_2018_Hamilton.pdf"

# Extract only the page 3 tables from the PDF with 'lattice' option:
tables = camelot.read_pdf(pdf_path, pages="3-3", flavor="lattice")

In [3]:
# Check all elements of 'tables' variable:
for table in tables:
    print(table.df)

                                                    0          1      2  \
0                                                      FULL-TIME          
1                                                            Men  Women   
2                                      Undergraduates                     
3               Degree-seeking, first-time \nfreshmen        217    263   
4                    Other first-year, degree-seeking                     
5                            All other degree-seeking        669    736   
6                                Total degree-seeking        886    999   
7   All other undergraduates enrolled \nin credit ...          0      1   
8                                Total undergraduates        886  1,000   
9                                            Graduate                     
10                         Degree-seeking, first-time                     
11                           All other degree-seeking                     
12   All other graduates 

<div style="text-align: justify">
The result of the 'tables' variable isn't very useful as it doesn't directly indicate how many tables were extracted. However, we know that page 3 of the PDF includes <strong>3 tables</strong> and the B1 table (the first table) is the table of interest. Let's investigate further!

In [4]:
# Check how many tables extracted from page 3, hoping to see '3' as an answer:
print(len(tables))

3


Since the number of elements inside 'tables' list is correct, the last step is to check how these elements are separated.

In [5]:
# Check the first element of the variable 'tables':
print(tables[0].df)

                                                    0          1      2  \
0                                                      FULL-TIME          
1                                                            Men  Women   
2                                      Undergraduates                     
3               Degree-seeking, first-time \nfreshmen        217    263   
4                    Other first-year, degree-seeking                     
5                            All other degree-seeking        669    736   
6                                Total degree-seeking        886    999   
7   All other undergraduates enrolled \nin credit ...          0      1   
8                                Total undergraduates        886  1,000   
9                                            Graduate                     
10                         Degree-seeking, first-time                     
11                           All other degree-seeking                     
12   All other graduates 

Indeed, the first element of the 'tables' list includes all data of the B1 table!!!

In [6]:
# Check if the extraction was accurate:
print(tables[0].parsing_report)

{'accuracy': 97.9, 'whitespace': 45.71, 'order': 1, 'page': 3}


In [7]:
# Assign the B1 pdf's table to a variable:
b1_table = tables[0].df

## Data Cleaning

In [8]:
# This is already a df:
type(b1_table)

pandas.core.frame.DataFrame

In [9]:
print(b1_table.shape)
b1_table

(14, 5)


,0,1,2,3,4
0,,FULL-TIME,,PART-TIME,
1,,Men,Women,Men,Women
2,Undergraduates,,,,
3,"Degree-seeking, first-time \nfreshmen",217,263,,
4,"Other first-year, degree-seeking",,,,
5,All other degree-seeking,669,736,,
6,Total degree-seeking,886,999,0,0
7,All other undergraduates enrolled \nin credit ...,0,1,2,9
8,Total undergraduates,886,"1,000",2,9
9,Graduate,,,,


In [10]:
# Set the correct column names:
b1_table.columns = ["studies_info", "men_fulltime", "women_fulltime", "men_parttime", "women_parttime"]

# Drop observations 0 and 1 (first two rows):
b1_table = b1_table.drop([0, 1], axis=0)
b1_table

,studies_info,men_fulltime,women_fulltime,men_parttime,women_parttime
2,Undergraduates,,,,
3,"Degree-seeking, first-time \nfreshmen",217,263,,
4,"Other first-year, degree-seeking",,,,
5,All other degree-seeking,669,736,,
6,Total degree-seeking,886,999,0,0
7,All other undergraduates enrolled \nin credit ...,0,1,2,9
8,Total undergraduates,886,"1,000",2,9
9,Graduate,,,,
10,"Degree-seeking, first-time",,,,
11,All other degree-seeking,,,,


<div style="text-align: justify">
A good progress has been made and the information of the original B1 pdf table seems to be shown here in a clear way.  
The next step is to interpret the missing values of the table. The table attempts to present information for two basic categories of students, undergraduates (bachelor) and graduates (higher education level). The empty values of second category probably means that Hamilton Collegege doesn't offer graduate programs or PHDs, at least at 2017-2018 where this study has been conducted.  
For this reason, it is reasonable to drop all these lines.

In [11]:
# Drop rows 9-13:
b1_table = b1_table.drop([9, 10, 11, 12, 13], axis=0)
b1_table

,studies_info,men_fulltime,women_fulltime,men_parttime,women_parttime
2,Undergraduates,,,,
3,"Degree-seeking, first-time \nfreshmen",217,263,,
4,"Other first-year, degree-seeking",,,,
5,All other degree-seeking,669,736,,
6,Total degree-seeking,886,999,0,0
7,All other undergraduates enrolled \nin credit ...,0,1,2,9
8,Total undergraduates,886,"1,000",2,9


<div style="text-align: justify">
The information of row number 2 should be passed in the column names and then dropped for better data interpretability. The abreviation "ug" will be passed instead of the whole word "undergraduate".

In [12]:
# Set the final column names:
b1_table.columns = ["studies_info", "ug_men_fulltime", "ug_women_fulltime", "ug_men_parttime", "ug_women_parttime"]

# Drop final row:
b1_table = b1_table.drop(2, axis=0)
b1_table

,studies_info,ug_men_fulltime,ug_women_fulltime,ug_men_parttime,ug_women_parttime
3,"Degree-seeking, first-time \nfreshmen",217,263,,
4,"Other first-year, degree-seeking",,,,
5,All other degree-seeking,669,736,,
6,Total degree-seeking,886,999,0,0
7,All other undergraduates enrolled \nin credit ...,0,1,2,9
8,Total undergraduates,886,"1,000",2,9


The emtpy cells left doesn't need further investigation and they should be replaced by zeros, because this is what they represent.

In [13]:
# Replace the empty values with 0:
b1_table = b1_table.replace('', 0)
b1_table

,studies_info,ug_men_fulltime,ug_women_fulltime,ug_men_parttime,ug_women_parttime
3,"Degree-seeking, first-time \nfreshmen",217,263,0,0
4,"Other first-year, degree-seeking",0,0,0,0
5,All other degree-seeking,669,736,0,0
6,Total degree-seeking,886,999,0,0
7,All other undergraduates enrolled \nin credit ...,0,1,2,9
8,Total undergraduates,886,"1,000",2,9


In [14]:
# Fixes the indexing:
b1_table = b1_table.reset_index(drop=True)
b1_table

,studies_info,ug_men_fulltime,ug_women_fulltime,ug_men_parttime,ug_women_parttime
0,"Degree-seeking, first-time \nfreshmen",217,263,0,0
1,"Other first-year, degree-seeking",0,0,0,0
2,All other degree-seeking,669,736,0,0
3,Total degree-seeking,886,999,0,0
4,All other undergraduates enrolled \nin credit ...,0,1,2,9
5,Total undergraduates,886,"1,000",2,9


In [15]:
# Replace commas wherever is needed to:
b1_table = b1_table.replace(",", "", regex=True)
b1_table

,studies_info,ug_men_fulltime,ug_women_fulltime,ug_men_parttime,ug_women_parttime
0,Degree-seeking first-time \nfreshmen,217,263,0,0
1,Other first-year degree-seeking,0,0,0,0
2,All other degree-seeking,669,736,0,0
3,Total degree-seeking,886,999,0,0
4,All other undergraduates enrolled \nin credit ...,0,1,2,9
5,Total undergraduates,886,1000,2,9


In [16]:
# Replace the line brakes wherever is needed to:
b1_table = b1_table.replace("\n", "", regex=True)
b1_table

,studies_info,ug_men_fulltime,ug_women_fulltime,ug_men_parttime,ug_women_parttime
0,Degree-seeking first-time freshmen,217,263,0,0
1,Other first-year degree-seeking,0,0,0,0
2,All other degree-seeking,669,736,0,0
3,Total degree-seeking,886,999,0,0
4,All other undergraduates enrolled in credit co...,0,1,2,9
5,Total undergraduates,886,1000,2,9


<div style="text-align: justify">
This is the final result and the table data seem to be tidy. The very last step is to ensure the table columns dtypes. Of course, this isn't a typical df where the rows represent observations. Here the rows represent undergraduate students' studies information categories. However, this isn't affecting the original intentions of this project in any way.

In [17]:
# The table contains wrong column dtypes:
b1_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   studies_info       6 non-null      object
 1   ug_men_fulltime    6 non-null      object
 2   ug_women_fulltime  6 non-null      object
 3   ug_men_parttime    6 non-null      object
 4   ug_women_parttime  6 non-null      object
dtypes: object(5)
memory usage: 372.0+ bytes


In [18]:
# Correct the columns dtype:
b1_table[
[
    "ug_men_fulltime", 
    "ug_women_fulltime", 
    "ug_men_parttime", 
    "ug_women_parttime"
]
] = b1_table[["ug_men_fulltime", "ug_women_fulltime", "ug_men_parttime", "ug_women_parttime"]].astype(int)
b1_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   studies_info       6 non-null      object
 1   ug_men_fulltime    6 non-null      int32 
 2   ug_women_fulltime  6 non-null      int32 
 3   ug_men_parttime    6 non-null      int32 
 4   ug_women_parttime  6 non-null      int32 
dtypes: int32(4), object(1)
memory usage: 276.0+ bytes


## Basic Statistics and Probability Theory Applications
<div style="text-align: justify">
This section is for the purpose of Statistics and Probability Theory, fulfilling the original intention of this project. The main goal is to demonstrate some basic knowledge of probability theory using Jupyter Notebook and Markdown syntax.

__<br>
What is the sample space of Hamilton College students? What are the favorable outcomes of being a male and being a female?__

<div style="text-align: justify">
The total sample space of Hamilton College students is $Ω = 1897$, whereas the favorable outcomes of being a male are $N_{M} = 888$ and the favorable outcomes of being a female are $N_{F} = 1009$.

__What are the probabilities of choosing a male and a female student from the sample space?__

<div style="text-align: justify">
From the classical definition of probability, the probability of choosing a male student out of the sample space is $P_{MALE}=\frac{N_{M}}{Ω}=0.4681$, whereas the corresponding probability of being a female is $P_{FEMALE}=\frac{N_{F}}{Ω}=0.5319$.

__What is the likelihood of being both a female and a freshman?__

This is the intersection, $P(Female∩Freshman)$ and equals to $263/1897$, so, $P(Female∩Freshman)=0.1386$.

__What is the probability of someone being a freshman?__

The probability of someone being a freshman is $P_{FRESHMAN}=0.253$.

__What is the likelihood of someone being a female, given that the same person is a freshman?__

This is the conditional probability:  
$$P(Female∣Freshman)=\frac{P(Female∩Freshman)}{P_{FRESHMAN}}=0.5478$$

**How was $P_{FRESHMAN}=0.253$ calculated?**

<div style="text-align: justify">
Well, this is an event union case where: 
$$P_{FRESHMAN}=P(Female∩Freshman)+P(Male∩Freshman)-P((Female∩Freshman)∩(Male∩Freshman))$$  
Since, $P((Female∩Freshman)∩(Male∩Freshman))=Ø$, because they are mutually exclusive events (a person can't be a male and a female simultaneously):  
$$P_{FRESHMAN}=\frac{263}{1897}+\frac{217}{1897}=0.253$$

__<div style="text-align: justify">
What is the probability of being a male, given that the same person is not a freshman? How is it compared with a not freshman student, given that he is a male?__

<div style="text-align: justify">
The number of male students who are not freshmen equals to $N_{MALENOTFRESHMAN}=671$. The probability can be quickly calculated by dividing this number with the sample space $Ω-N_{FRESHMAN}$, but, let's find the same probability applying some probability theory in the same way as before. The probability that should be calculated is the conditional probability $P(Male∣NotFreshman)$:  
$$P(Male∣NotFreshman)=\frac{P(Male∩NotFreshman)}{P_{NOTFRESHMAN}}=\frac{\frac{671}{1897}}{\frac{1417}{1897}}=0.4735$$

<div style="text-align: justify">
To compare $P(Male∣NotFreshman)$ with $P(NotFreshman∣Male)$ the Bayes' theorem will be used:  
$$P(NotFreshman∣Male)=\frac{P(Male∣NotFreshman)P_{NOTFRESHMAN}}{P_{MALE}}=\frac{0.4735*0.7469}{0.4681}=0.7555$$